<a href="https://colab.research.google.com/github/jpscard/I2A2_AI_Industry_Desafios/blob/main/I2A2_ABDI_Turma_4_Desafio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## *I2A2-ABDI - Turma 4 - Desafio 3*
### *Aluno:João Paulo da Silva Cardoso*

##**Enunciado do desafio**

O Desafio de Previsão de Demanda de Itens de Loja no Kaggle envolve prever dados de vendas futuras para 10 lojas diferentes e 50 itens. O objetivo desta competição é prever 3 meses de dados de vendas em nível de item em diferentes locais de loja.

**A tarefa é:**

Tomando como base a framework CRISP-DM:a. Fazer a análise exploratória dos dados
b. Realizar a criação dos modelos para predizer as vendas das lojas/produtos e datas conforme constante no arquivo test.csv
c. Preparar e submeter o arquivo “submission.csv”, utilizando a opção “late submission”.

O Estudo deve seguir a metodologia CRISP-DS, descrevendo as etapas realizadas. Ao final, coloquem também seus comentários. Como sempre, a entrega é obrigatória e vocês tem até o dia 05/06 às 23h59.

### 0. Integração com o Kaggle

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F9999%2F868225%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240606%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240606T022354Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6292424383944ba8636c169fe5cce20adc237185de3aa60ec9e1b00a254e7a6c3b5fc7a384802af95039c576b7e3158211a590f68e77d619206ff70417f974745dae5debb199438bb01ff58dd1787a63c20800b14d1147f5be6571a6bf9129ad9b1ba4c7ae6b479b220d88b730b244945c1d4d7ee8e67201cdeab01bad25b0b4351b496ad4582d6c30bb70efb1c13b5bc1f006d01adf03b2709d53e298655d76014a9940dcb92ea19ec90ae1ff39aa41dc0a7363fe7d631582bd0633e8f304b0eacda24cff6e0c8b498fc29c0444027945dd7ba8772b966165ec78a04c536953f152aac3ec376c65dcb754a29dc541b1af7b877f496ec8088439d6a02e793af0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 3454726 bytes downloaded
Downloaded and uncompressed: 
Data source import complete.


### 1. Entendimento do Negócio

O problema envolve a previsão de vendas futuras para 10 lojas e 50 itens ao longo de vários anos. O objetivo é prever a demanda diária para cada combinação de loja e item.

### 2. Entendimento dos dados

**Descrição do Conjunto de Dados**

O objetivo desta competição é prever 3 meses de dados de vendas de itens em diferentes localizações de lojas.

**Descrição dos Arquivos**
- **train.csv**: Dados de treinamento
- **test.csv**: Dados de teste (Observação: a divisão Público/Privado é baseada no tempo)
- **sample_submission.csv**: um arquivo de submissão de exemplo no formato correto

**Campos de Dados**
- **date**: Data dos dados de venda. Não há efeitos de feriado ou fechamentos de loja.
- **store**: ID da loja
- **item**: ID do item
- **sales**: Número de itens vendidos em uma loja em uma data específica.

### 3. Preparação dos Dados

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import time
import datetime
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score  # Importa as métricas MSE e R² da biblioteca scikit-learn

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'test.csv', 'train.csv']


In [ ]:
import pandas as pd  # Importa a biblioteca pandas e a renomeia como pd

# Lê o arquivo 'train.csv' e armazena os dados em um DataFrame chamado 'data'
data = pd.read_csv('../input/train.csv')

# Lê o arquivo 'test.csv', define a coluna 'id' como índice e armazena os dados em um DataFrame chamado 'test_data'
test_data = pd.read_csv('../input/test.csv', index_col='id')

# Imprime o valor mínimo e máximo da coluna 'sales' no DataFrame 'data'
print(data['sales'].min(), data['sales'].max())

0 231


In [ ]:
# Convertendo a coluna 'date' para o tipo datetime no DataFrame 'data'
data['date'] = pd.to_datetime(data['date'])

# Convertendo a coluna 'date' para o tipo datetime no DataFrame 'test_data'
test_data['date'] = pd.to_datetime(test_data['date'])

# Criando uma nova coluna 'month' no DataFrame 'data' para armazenar o mês da data
data['month'] = data['date'].dt.month

# Criando uma nova coluna 'day' no DataFrame 'data' para armazenar o dia da semana da data (0 para segunda-feira, 6 para domingo)
data['day'] = data['date'].dt.dayofweek

# Criando uma nova coluna 'year' no DataFrame 'data' para armazenar o ano da data
data['year'] = data['date'].dt.year

# Criando uma nova coluna 'month' no DataFrame 'test_data' para armazenar o mês da data
test_data['month'] = test_data['date'].dt.month

# Criando uma nova coluna 'day' no DataFrame 'test_data' para armazenar o dia da semana da data (0 para segunda-feira, 6 para domingo)
test_data['day'] = test_data['date'].dt.dayofweek

# Criando uma nova coluna 'year' no DataFrame 'test_data' para armazenar o ano da data
test_data['year'] = test_data['date'].dt.year


In [ ]:
# Lista das colunas a serem usadas como variáveis independentes
x_columns = ['store', 'item', 'month', 'day', 'year']

# Selecionando as colunas especificadas como variáveis independentes e armazenando em X
X = data[x_columns]

# Selecionando a coluna na posição 3 (presumivelmente a coluna 'sales') como variável dependente e armazenando em Y
Y = data.iloc[:, 3]

In [ ]:
# Dividindo os conjuntos de dados X e Y em conjuntos de treinamento e teste
# O parâmetro test_size=.2 especifica que 20% dos dados serão usados para teste
# Os conjuntos de treinamento e teste são atribuídos a train_X, test_X, train_Y e test_Y, respectivamente
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=.2)

### 4. Modelagem

In [ ]:
# Lista de chaves (nomes dos regressores)
key = ['RandomForestRegressor', 'XGBRegressor', 'LightGBM']

# Lista de valores (os regressores instanciados com o parâmetro random_state=51)
value = [RandomForestRegressor(random_state=51), XGBRegressor(random_state=51), lgb.LGBMRegressor(random_state=51)]

# Cria um dicionário onde as chaves são os nomes dos regressores e os valores são os regressores instanciados
models = dict(zip(key, value))

In [ ]:
# Lista vazia para armazenar o MSE, RMSE e R²
mse1 = []
rmse1 = []
r21 = []

# Loop sobre os modelos no dicionário models
for name, algo in models.items():
    model = algo
    model.fit(train_X, train_Y)  # Treina o modelo com os dados de treinamento
    predict = model.predict(test_X)  # Faz previsões com o modelo nos dados de teste
    mse = mean_squared_error(test_Y, predict)  # Calcula o MSE
    mse1.append(mse)  # Armazena o MSE na lista mse1
    rmse = np.sqrt(mean_squared_error(test_Y, predict))  # Calcula o RMSE
    rmse1.append(rmse)  # Armazena o RMSE na lista rmse1
    r2 = r2_score(test_Y, predict)  # Calcula o R²
    r21.append(r2)  # Armazena o R² na lista r21
    print(name, '\t', mse, '\t', rmse, '\t', r2)  # Imprime os resultados

# Cria um dicionário com os resultados
result = {
    'Model': key,
    'Mean Square Error': mse1,
    'Root Mean Square Error': rmse1,
    'R-Squared': r21
}

# Cria um DataFrame com os resultados
model_results = pd.DataFrame(result)

RandomForestRegressor 	 69.05883900168013 	 8.310164799911018 	 0.9162091998521393
XGBRegressor 	 58.37433536623514 	 7.640309899882015 	 0.9291729727990734
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013364 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 730400, number of used features: 5
[LightGBM] [Info] Start training from score 52.271495
LightGBM 	 68.48692242556314 	 8.275682595747806 	 0.9169031204019693


### 5. Avaliação

In [ ]:
# Classificação dos resultados com base no Root Mean Square Error (RMSE) em ordem crescente
model_results.sort_values('Root Mean Square Error', ascending=True)

,Model,Mean Square Error,Root Mean Square Error,R-Squared
1,XGBRegressor,58.374335,7.640310,0.929173
2,LightGBM,68.486922,8.275683,0.916903
0,RandomForestRegressor,69.058839,8.310165,0.916209
